In [1]:
from copy import deepcopy
import numpy.linalg as la
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split

from src.models.estimator import RFClassifier, relu
from src.models.weights import V1_weights, classical_weights, sensilla_weights
from src.data.load_dataset import load_mnist, load_kmnist, generate_frequency_detection, generate_frequency_XOR

In [ ]:
# load dataset
train_loader, val_loader, test_loader = load_kmnist()
train, test = train_loader.dataset.dataset, test_loader.dataset
train, train_labels = train.data.numpy(), train.targets.numpy()
X_test, y_test = test.data.numpy(), test.targets.numpy()

X_train, X_val, y_train, y_val = train_test_split(train, train_labels, train_size=0.5, stratify=train_labels)

In [ ]:
# train params
h = 1000
bias = 0
nonlinearity = relu
scale = 1
clf = LinearSVC(tol=1e-4, max_iter=1000)

In [ ]:
%%time
# define the V1 model
kwargs = {'size':5, 'spatial_freq': 2, 'center': None, 'scale': scale}
v1_clf = RFClassifier(h, V1_weights, bias, nonlinearity, deepcopy(clf), kwargs)
v1_clf.fit(X_train, y_train)
v1_clf.score(X_test, y_test)

In [ ]:
%%time
# define the classical model
kwargs = {'scale': scale}
classical_clf = RFClassifier(h, classical_weights, bias, nonlinearity, deepcopy(clf), kwargs)
classical_clf.fit(X_train, y_train)
classical_clf.score(X_test, y_test)

### tseries classification

In [107]:
# train params
h = 250
bias = 0
nonlinearity = relu
scale = 1
clf = LinearSVC(tol=1e-4, max_iter=1000)

num_samples, sampling_rate, duration, freq, snr, seed = 7000, 2000, 0.1, 50, 0.8, 5
X, y = generate_frequency_detection(num_samples, sampling_rate, freq, duration, snr, seed)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, stratify=y)

In [110]:
# define the sensilla model
kwargs = {'sampling_rate': sampling_rate, 'duration':duration, 'lowcut': 20, 'highcut': 80, 'decay_coef':0.05, 'scale': scale}
mech_clf = RFClassifier(h, sensilla_weights, bias, nonlinearity, deepcopy(clf), kwargs)
mech_clf.fit(X_train, y_train)
1 - mech_clf.score(X_test, y_test)

/home/bpandey/anaconda3/envs/random_features/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


0.0007142857142856673

In [105]:
# define the classical model
kwargs = {'scale':scale}
classical_clf = RFClassifier(h, classical_weights, bias, nonlinearity, deepcopy(clf), kwargs)
classical_clf.fit(X_train, y_train)
1 - classical_clf.score(X_test, y_test)

/home/bpandey/anaconda3/envs/random_features/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


0.0485714285714286

In [ ]:
kwargs = {'sampling_rate': 2000, 'duration':0.1, 'lowcut': 10, 'highcut': 20, 'decay_coef':np.inf, 'scale': scale}
W = sensilla_weights(h, 200, **kwargs)
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(W[0])
plt.xticks([0, 50, 100, 150, 200])
ax.set_xticklabels([0, 25, 50, 75, 100])

### frequency XOR

In [91]:
# train params
h = 14
bias = 0
nonlinearity = relu
scale = 1
clf = LinearSVC(tol=1e-4, max_iter=1000)

num_samples, sampling_rate, duration, freq1, freq2, snr, seed = 7000, 2000, 0.1, 50, 80, 0.8, 5
X, y = generate_frequency_XOR(num_samples, sampling_rate, freq1, freq2, duration, snr, seed=None, shuffle=False)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, stratify=y)

In [92]:
# define the sensilla model
kwargs = {'sampling_rate': sampling_rate, 'duration':duration, 'lowcut': 30, 'highcut': 130, 'decay_coef':0.05, 'scale': scale}
mech_clf = RFClassifier(h, sensilla_weights, bias, nonlinearity, deepcopy(clf), kwargs)
mech_clf.fit(X_train, y_train)
1 - mech_clf.score(X_test, y_test)

/home/bpandey/anaconda3/envs/random_features/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


0.2014285714285714

In [93]:
# define the classical model
kwargs = {'scale':scale}
classical_clf = RFClassifier(h, classical_weights, bias, nonlinearity, deepcopy(clf), kwargs)
classical_clf.fit(X_train, y_train)
1 - classical_clf.score(X_test, y_test)

/home/bpandey/anaconda3/envs/random_features/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


0.4535714285714286

### test that joblib helps with parallel runs

In [6]:
# load dataset
train_loader, val_loader, test_loader = load_mnist()
train, test = train_loader.dataset.dataset, test_loader.dataset
train, train_labels = train.data.numpy(), train.targets.numpy()
X_test, y_test = test.data.numpy(), test.targets.numpy()

X_train, X_val, y_train, y_val = train_test_split(train, train_labels, train_size=0.99, stratify=train_labels)

In [7]:
# train params
h = 1000
bias = 0
nonlinearity = relu
scale = 1
clf = LinearSVC(tol=1e-4, max_iter=1000)

In [8]:
%%time
# define the V1 model
kwargs = {'size':5, 'spatial_freq': 2, 'center': None, 'scale': scale}
# v1_clf = RFClassifier(h, V1_weights, bias, nonlinearity, deepcopy(clf), kwargs)
# v1_clf.fit(X_train, y_train)
# v1_clf.score(X_test, y_test)

CPU times: user 19 µs, sys: 11 µs, total: 30 µs
Wall time: 9.78 µs


In [9]:
%%time
from joblib import Parallel, delayed, parallel_backend
n = 5
kwargs = {'size':5, 'spatial_freq': 2, 'center': None, 'scale': scale}
RFclassifiers = [RFClassifier(h, V1_weights, bias, nonlinearity, deepcopy(clf), kwargs) for i in range(n)]
with parallel_backend('threading', n_jobs=6):
    Parallel()(delayed(RFclf.fit)(X_train, y_train) for RFclf in RFclassifiers)
    
# Parallel(n_jobs=10)(delayed(RFclf.fit)(X_train, y_train) for RFclf in RFclassifiers)
print([RFclf.score(X_test, y_test) for RFclf in RFclassifiers ])

/home/bpandey/anaconda3/envs/random_features/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


[0.9762, 0.9777, 0.9813, 0.982, 0.9795]
CPU times: user 54min 52s, sys: 59.1 s, total: 55min 51s
Wall time: 10min 52s


/home/bpandey/anaconda3/envs/random_features/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


In [ ]:
%%time
RFclassifiers = [RFClassifier(h, V1_weights, bias, nonlinearity, deepcopy(clf), kwargs) for i in range(n)]
for RFclf in RFclassifiers:
    RFclf.fit(X_train, y_train)
print([RFclf.score(X_test, y_test) for RFclf in RFclassifiers ])